In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split

2023-04-19 22:51:14.575393: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
colour_ints = {"blue_cone": 0,
               "yellow_cone": 1,
               "orange_cone": 2,
               "large_orange_cone": 3
               }

In [6]:
pics = "/Users/stephenmesser/Desktop/FS-AI/extracted_cones"
directory = os.fsdecode(pics)

all_imgs = []
all_labels = []

for file_name in os.listdir(directory):

  if 'cone' in file_name and 'unknown' not in file_name:

    # add image to array
    img = cv2.imread(f'{pics}/{file_name}')
    res_image = cv2.resize(img, dsize=(50, 50))
    all_imgs.append(res_image)

    # add label to array
    all_labels.append(colour_ints[file_name.split('_cone_')[0] + '_cone'])

all_imgs = np.array(all_imgs)
print(all_imgs.shape)

(26508, 50, 50, 3)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.subplot(2,2,1)
plt.imshow(all_imgs[0], cmap=plt.get_cmap('gray'))
plt.subplot(2,2,2)
plt.imshow(all_imgs[1], cmap=plt.get_cmap('gray'))
plt.subplot(2,2,3)
plt.imshow(all_imgs[2], cmap=plt.get_cmap('gray'))
plt.subplot(2,2,4)
plt.imshow(all_imgs[61], cmap=plt.get_cmap('gray'))

#show the plot
plt.show()

In [7]:
#print(all_imgs[0])
# SCALING
all_imgs = all_imgs / 255
#print(all_imgs[0])

# ONE-HOT ENCODING
all_labels = np_utils.to_categorical(all_labels)
#print(all_labels)

In [12]:
num_classes = all_labels.shape[1]

In [8]:
reviews_train, reviews_test, y_train, y_test = train_test_split(all_imgs, all_labels, test_size=0.20)

In [9]:
len(reviews_train)

21206

In [24]:
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(50, 50, 3), activation='relu'))
    model.add(MaxPooling2D(strides=(5, 5)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    #Compile/train model
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model


In [25]:
#define the model
model = cnn_model()
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 46, 46, 32)        2432      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 9, 9, 32)         0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 9, 9, 32)          0         
                                                                 
 flatten_6 (Flatten)         (None, 2592)              0         
                                                                 
 dense_11 (Dense)            (None, 128)               331904    
                                                                 
 dense_12 (Dense)            (None, 4)                 516       
                                                      

In [26]:
#train the model
training = model.fit(reviews_train, y_train, validation_split = 0.2, epochs=6, batch_size=200)
#Evaluation of the model
scores = model.evaluate(reviews_test, y_test, verbose=0)
print("CNN Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/6
85/85 [==============================] - 10s 111ms/step - loss: 0.8453 - accuracy: 0.7294 - val_loss: 0.5331 - val_accuracy: 0.8225
Epoch 2/6
85/85 [==============================] - 9s 107ms/step - loss: 0.4535 - accuracy: 0.8570 - val_loss: 0.3542 - val_accuracy: 0.8956
Epoch 3/6
85/85 [==============================] - 9s 103ms/step - loss: 0.3416 - accuracy: 0.8978 - val_loss: 0.2917 - val_accuracy: 0.9222
Epoch 4/6
85/85 [==============================] - 9s 104ms/step - loss: 0.2900 - accuracy: 0.9122 - val_loss: 0.2439 - val_accuracy: 0.9399
Epoch 5/6
85/85 [==============================] - 9s 107ms/step - loss: 0.2511 - accuracy: 0.9257 - val_loss: 0.2235 - val_accuracy: 0.9394
Epoch 6/6
85/85 [==============================] - 9s 107ms/step - loss: 0.2323 - accuracy: 0.9311 - val_loss: 0.2005 - val_accuracy: 0.9470
CNN Accuracy: 94.19%


In [27]:
model.save('cones_model5050_stri55.keras')